In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import json, os, warnings

warnings.filterwarnings("ignore")
sns.set(style="whitegrid", font_scale=1.05)
OUT = "outputs"
os.makedirs(OUT, exist_ok=True)

# ---------------------------------------------------------------
# LOAD DATA
# ---------------------------------------------------------------
fg = pd.read_csv("fear_greed_index.csv")
trades = pd.read_csv("historical_data.csv")
print("Loaded shapes FG, trades:", fg.shape, trades.shape)

# ---------------------------------------------------------------
# FEAR-GREED CLEANING
# ---------------------------------------------------------------
fg.rename(columns={'value': 'index_value'}, inplace=True)

def parse_fg_timestamp(series):
    s = pd.to_numeric(series, errors='coerce')
    unit = 's' if s.median() < 1e11 else 'ms'
    return pd.to_datetime(s, unit=unit, errors='coerce', utc=True)

fg['timestamp'] = parse_fg_timestamp(fg['timestamp'])
fg = fg.dropna(subset=['timestamp'])
fg = fg[['timestamp', 'index_value', 'classification']].drop_duplicates()
print(f"FG cleaned rows: {len(fg)}")

# ---------------------------------------------------------------
# TRADES CLEANING
# ---------------------------------------------------------------
ts_cols = [c for c in trades.columns if 'timestamp' in c.lower()]
print("Candidate timestamp columns found:", ts_cols)

best_ts = None
max_unique = 0
for c in ts_cols:
    unique_days = pd.to_datetime(trades[c], errors='coerce').dt.date.nunique()
    print(f"→ {c}: {unique_days} unique trading days")
    if unique_days > max_unique:
        best_ts, max_unique = c, unique_days

ts_col = best_ts or 'timestamp updated'
print(f"Best timestamp column selected: {ts_col} ({max_unique} unique days)")

# --- FIXED robust IST parsing ---
def smart_parse_timestamp(series):
    s = pd.to_datetime(series, errors='coerce', utc=False, infer_datetime_format=True)

    if s.isna().mean() > 0.3:
        # Try multiple common formats
        for fmt in [
            "%d-%m-%Y %H:%M:%S", "%d/%m/%Y %H:%M:%S",
            "%Y-%m-%d %H:%M:%S", "%d-%m-%Y %I:%M:%S %p",
            "%d-%m-%Y", "%Y/%m/%d %H:%M"
        ]:
            s2 = pd.to_datetime(series, format=fmt, errors="coerce")
            if s2.notna().sum() > s.notna().sum():
                s = s2

    # Localize safely without `errors`
    try:
        s = s.dt.tz_localize("Asia/Kolkata", nonexistent="shift_forward", ambiguous="NaT")
    except Exception:
        # If already tz-aware, skip localization
        pass

    return s.dt.tz_convert("UTC")

    # Localize safely without `errors`
    try:
        s = s.dt.tz_localize("Asia/Kolkata", nonexistent="shift_forward", ambiguous="NaT")
    except Exception:
        # If already tz-aware, skip localization
        pass

    return s.dt.tz_convert("UTC")

    # Localize to IST safely
    s = s.dt.tz_localize("Asia/Kolkata", nonexistent="shift_forward", ambiguous="NaT", errors="coerce")
    return s.dt.tz_convert("UTC")

trades['timestamp_final'] = smart_parse_timestamp(trades[ts_col])
trades = trades.dropna(subset=['timestamp_final'])

for c in ['Execution Price','Size Tokens','Size USD','Closed PnL','Fee']:
    if c in trades.columns:
        trades[c] = pd.to_numeric(trades[c], errors='coerce')

print(f"Trade timestamp range: {trades['timestamp_final'].min()} → {trades['timestamp_final'].max()}")
print(f"FG timestamp range: {fg['timestamp'].min()} → {fg['timestamp'].max()}")
print(f"Unique trade days detected: {trades['timestamp_final'].dt.date.nunique()}")

# ---------------------------------------------------------------
# NEAREST TIMESTAMP MERGE
# ---------------------------------------------------------------
print("\nMerging each trade with nearest Fear-Greed timestamp (within 1 day)...")

trades = trades.sort_values('timestamp_final')
fg = fg.sort_values('timestamp')

merged = pd.merge_asof(
    trades, fg,
    left_on='timestamp_final',
    right_on='timestamp',
    direction='nearest',
    tolerance=pd.Timedelta('1D')
)

merged = merged.dropna(subset=['index_value', 'classification'])
print(f"Final merged rows: {len(merged)}")
print(f"Merged timestamp range: {merged['timestamp_final'].min()} → {merged['timestamp_final'].max()}")

# ---------------------------------------------------------------
# SENTIMENT SUMMARY
# ---------------------------------------------------------------
sent_summary = merged.groupby('classification').agg(
    trades=('timestamp_final','count'),
    avg_pnl=('Closed PnL','mean'),
    median_pnl=('Closed PnL','median'),
    avg_volume=('Size USD','mean'),
    avg_exec=('Execution Price','mean')
).reset_index()

sent_summary.to_csv(f"{OUT}/sentiment_summary.csv", index=False)
print("\nSentiment summary:\n", sent_summary)

# ---------------------------------------------------------------
# STATISTICAL TESTS
# ---------------------------------------------------------------
groups = [g['Closed PnL'].dropna() for _, g in merged.groupby('classification')]
kw_res = stats.kruskal(*groups) if len(groups) > 1 else None
if kw_res:
    print(f"\nKruskal-Wallis p-value: {kw_res.pvalue:.4f}")

# ---------------------------------------------------------------
# VOLATILITY ANALYSIS
# ---------------------------------------------------------------
merged['PnL_roll_std_500'] = merged['Closed PnL'].rolling(500, min_periods=20).std()
vol_by_sent = merged.groupby('classification')['PnL_roll_std_500'].mean().reset_index()
vol_by_sent.to_csv(f"{OUT}/volatility_by_sentiment.csv", index=False)

# ---------------------------------------------------------------
# MODEL: Predict PnL Using Sentiment
# ---------------------------------------------------------------
model_df = merged.dropna(subset=['Closed PnL','index_value']).copy()
model_df['hour'] = model_df['timestamp_final'].dt.hour
model_df = pd.get_dummies(model_df, columns=['classification'], drop_first=True)
X = model_df.select_dtypes(include=[np.number]).drop(columns=['Closed PnL'], errors='ignore')
y = model_df['Closed PnL']

if len(X) >= 500:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    rmse = np.sqrt(np.mean((preds - y_test)**2))
    perm = permutation_importance(rf, X_test, y_test, n_repeats=20, random_state=42)
    imp = pd.Series(perm.importances_mean, index=X_test.columns).sort_values(ascending=False)
    imp.head(15).to_csv(f"{OUT}/feature_importances.csv")
    print(f"\nRandom Forest RMSE: {rmse:.4f}")
else:
    print("\nNot enough data to train a robust model.")

# ---------------------------------------------------------------
# VISUALIZATIONS
# ---------------------------------------------------------------
plt.figure(figsize=(14,5))
sns.lineplot(x='timestamp_final', y='index_value', data=merged, color='royalblue', linewidth=1.5)
plt.title("Fear-Greed Index (Nearest Timestamp Merge)", fontsize=14, fontweight='bold')
plt.xlabel("Timestamp"); plt.ylabel("Index Value")
plt.tight_layout()
plt.savefig(f"{OUT}/fig_timeseries_index.png"); plt.close()

plt.figure(figsize=(14,5))
sns.lineplot(x='timestamp_final', y='Closed PnL', data=merged, color='seagreen', linewidth=1.5)
plt.title("Trader Closed PnL Over Time", fontsize=14, fontweight='bold')
plt.xlabel("Timestamp"); plt.ylabel("Closed PnL")
plt.tight_layout()
plt.savefig(f"{OUT}/fig_timeseries_pnl.png"); plt.close()

plt.figure(figsize=(8,5))
sns.regplot(x='index_value', y='Closed PnL', data=merged, scatter_kws={'alpha':0.3}, line_kws={'color':'crimson'})
plt.title("Fear-Greed Index vs Closed PnL", fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{OUT}/fig_scatter_index_vs_pnl.png"); plt.close()

plt.figure(figsize=(8,5))
sns.boxplot(x='classification', y='Closed PnL', data=merged, palette='viridis')
plt.title("PnL Distribution by Sentiment", fontsize=13, fontweight='bold')
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(f"{OUT}/fig_box_pnl_by_sentiment.png"); plt.close()

# ---------------------------------------------------------------
# INSIGHTS SUMMARY
# ---------------------------------------------------------------
corr_val = merged['index_value'].corr(merged['Closed PnL'])
ins = [
    f"Rows merged: {len(merged)}",
    f"Correlation (Fear-Greed vs Closed_PnL): {corr_val:.4f}",
]
if kw_res:
    ins.append(f"Kruskal-Wallis p-value: {kw_res.pvalue:.4e}")
if len(X) >= 500:
    ins.append(f"Random Forest RMSE: {rmse:.3f}")
ins.append("Sentiment impact on volatility saved to outputs/volatility_by_sentiment.csv")
ins.append("Visualizations saved to outputs/*.png")

# Save insights
open(f"{OUT}/insights.txt", "w").write("\n".join(ins))

# Save merged data to CSV for verification
merged.to_csv(f"{OUT}/merged_trades_with_sentiment.csv", index=False)

# Confirm in console
print("\n".join(ins))
print(f"\nMerged file saved to: {OUT}/merged_trades_with_sentiment.csv")
print("\nAnalysis complete. Outputs stored in the 'outputs' folder.")


Loaded shapes FG, trades: (2644, 6) (211224, 17)
FG cleaned rows: 2644
Candidate timestamp columns found: ['Timestamp IST', 'Timestamp', 'timestamp updated']
→ Timestamp IST: 188 unique trading days
→ Timestamp: 1 unique trading days
→ timestamp updated: 7 unique trading days
Best timestamp column selected: Timestamp IST (188 unique days)
Trade timestamp range: 2023-04-30 18:30:00+00:00 → 2025-04-30 18:30:00+00:00
FG timestamp range: 2018-02-01 05:30:00+00:00 → 2025-05-02 05:30:00+00:00
Unique trade days detected: 480

Merging each trade with nearest Fear-Greed timestamp (within 1 day)...
Final merged rows: 211224
Merged timestamp range: 2023-04-30 18:30:00+00:00 → 2025-04-30 18:30:00+00:00

Sentiment summary:
   classification  trades    avg_pnl  median_pnl   avg_volume      avg_exec
0   Extreme Fear   21400  34.537862         0.0  5349.731843   7054.795108
1  Extreme Greed   39992  67.892861         0.0  3112.251565   6082.195865
2           Fear   61837  54.290400         0.0  7816.